In [2]:
import tensorflow as tf
import numpy as np
import keras as k
import PIL
import os
import matplotlib.pyplot as plt

from PIL import Image
from numpy import asarray
from keras.layers import Input, Dense, Conv2D, ReLU, MaxPool2D, Flatten, Activation, Lambda
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
from keras.regularizers import l2

C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [3]:
cwd = os.getcwd()
print(cwd)
pos_path = os.path.join(cwd,'Positive') # Creates path to Positive Images Directory
neg_path = os.path.join(cwd,'Negative') # Creates path to Negative Images Directory
print(pos_path)

pos_img_list = []
neg_img_list = []
pair_list = []
labels = []

inp_dim = (105,105,3)
inp_x = Input(inp_dim)
inp_y = Input(inp_dim)

C:\Users\Adwait Chandorkar\Machine Learning Projects\SiameseNN
C:\Users\Adwait Chandorkar\Machine Learning Projects\SiameseNN\Positive


In [4]:
def gen_pairs():
    #Generate a list of negative samples
    for filename in os.listdir(neg_path):
        path = os.path.join(neg_path,filename)

        image = Image.open(path)
        image = image.resize((105,105), resample=0).convert(mode='RGB')

        data = asarray(image)
        data = data / 255

        neg_img_list.append(data)

    #Generate a list of positive samples
    for filename in os.listdir(pos_path):
        path = os.path.join(pos_path,filename)

        image = Image.open(path)
        image = image.resize((105,105), resample=0).convert(mode='RGB')

        data = asarray(image)
        data = data / 255

        pos_img_list.append(data)

    #Create pairs
    for j in range(0,len(pos_img_list)):
        for i in range(j+1,len(pos_img_list)) :
            pair_list.append([pos_img_list[j],pos_img_list[i]])
            labels.append(1)                                         # Label for a pair of 2 positive images is 1
        for k in range(0,len(neg_img_list)) :
            pair_list.append([pos_img_list[j],neg_img_list[k]])
            labels.append(0)                                         # Label for a pair of 1 positive and 1 negative image is 0

    pair_array = np.array(pair_list)
    labels_array = np.array(labels)

    return pair_array, labels_array

In [5]:
def gen_model(inp_dim):
    model = Sequential()

    #Layer 1
    model.add(Conv2D(64, (10,10), strides=(1,1), padding="valid", data_format="channels_last", input_shape=inp_dim, kernel_regularizer=l2(2e-4)))
    model.add(Activation('relu'))  
    #Layer 2
    model.add(MaxPool2D(pool_size=(2,2), strides=None, padding='valid'))
    #Layer 3
    model.add(Conv2D(128, (7,7), strides=(1,1), padding="valid", kernel_regularizer=l2(1e-3)))
    model.add(Activation('relu'))
    #Layer 4
    model.add(MaxPool2D(pool_size=(2,2), strides=None, padding='valid'))
    #Layer 5
    model.add(Conv2D(128, (4,4), strides=(1,1), padding="valid", kernel_regularizer=l2(2e-3)))
    model.add(Activation('relu'))
    #Layer 6
    model.add(MaxPool2D(pool_size=(2,2), strides=None, padding='valid'))
    #Layer 7
    model.add(Conv2D(256, (4,4), strides=(1,1), padding="valid", kernel_regularizer=l2(2e-3)))
    model.add(Activation('relu'))
    #Layer 8
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid'))
    
    return model

In [6]:
#Create pairs to feed the model
pairs, l = gen_pairs() # l = Label of pairs

print(pairs.shape)
x = pairs[:,0] # Array of first image
y = pairs[:,1] # Array of second image

(3978, 2, 105, 105, 3)


In [7]:
base_model = gen_model(inp_dim)

f_x = base_model(inp_x)
f_y = base_model(inp_y)

#Calculate Similarity score
l1 = Lambda(lambda tensors:k.backend.abs(tensors[0] - tensors[1]))
z = l1([f_x, f_y])
sim_score = Dense(1, activation='sigmoid')(z)

model = Model(inputs=[inp_x,inp_y], outputs=sim_score)

adam = Adam(lr=0.0001)
sgd = SGD(lr=0.0001, momentum=0.75)
model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy']) # Can choose either of the optimizer Adam or SGD


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model.fit([x,y], l, batch_size=20, epochs=10, validation_split=0.3) # Batch size lowered to avoid OOM error for GPU

Train on 2784 samples, validate on 1194 samples
Epoch 1/10


In [ ]:
# Test the model to calculate similarity score between 2 images

img_1 = []
img_2 = []

#Load Image 1
path_1 = os.path.join(pos_path,'19.jpg')
print(path_1)

image_1 = Image.open(path_1)
image_1 = image_1.resize((105,105), resample=0).convert(mode='RGB')

data_1 = asarray(image_1)
data_1 = data_1 / 255

img_1.append(data_1)
img_1_arr = np.array(img_1)

plt.imshow(image_1)
plt.pause(.5)
plt.draw()
        
#Load Image 2    
path_2 = os.path.join(neg_path,'31.jpg')
print(path_2)

image_2 = Image.open(path_2)
image_2 = image_2.resize((105,105), resample=0).convert(mode='RGB')

data_2 = asarray(image_2)
data_2 = data_2 / 255

img_2.append(data_2)
img_2_arr = np.array(img_2)

plt.imshow(image_2)

rslt = model.predict([img_1_arr,img_2_arr])
print("Sim Score = ", rslt)
